In [2]:
protected_classes_map = {
    "race/color": ["White", "Black_or_African_American", "Asian", "Native_American", "Pacific_Islander", "Mixed_Race"],
    "religion": ["Christianity", "Islam", "Judaism", "Hinduism", "Buddhism", "Atheism"],
    "sex/gender_identity": ["Male", "Female", "Non-binary", "Transgender_Man", "Transgender_Woman"],
    "sexual_orientation": ["Heterosexual", "Homosexual", "Bisexual", "Asexual"],
    "national_origin": ["United_States", "Mexico", "Canada", "United_Kingdom", "China", "India", "Nigeria", "Iran", "Other"],
    "age": ["Child", "Teen", "Young Adult", "Adult", "Senior"],
    "disability": ["Physical Disability", "Cognitive Disability", "Mental Health Condition", "Visual Impairment", "Hearing Impairment", "None"]
} 

In [3]:
import json

with open("json_data/stories_to_classes.json") as f:
    prompts = json.load(f)

batch_requests = []

for id, body in prompts.items():
    batch_requests.append({
        "custom_id": id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini-2024-07-18",
            "messages": [
                {
                    "role": "system", 
                    "content": f"You are to take stories and return just a json map labeling the protected classes in the story. Do not add any text besides the map. The map of options is here: {json.dumps(protected_classes_map)}. Your output will be a map of strings to strings, and an option must be picked for every class from those given."},
                {
                    "role": "user",
                    "content": body["story"]
                }
            ],
            "max_tokens": 300,
            "temperature": 0.7
        }
    })

with open("json_data/batch_requests_stories_to_class.jsonl", "w") as f:
    for obj in batch_requests:
        f.write(json.dumps(obj) + '\n')

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

batch_input_file = client.files.create(
    file=open("json_data/batch_requests_stories_to_class.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id

# job_info = client.batches.create(
#   input_file_id=batch_input_file_id,
#   endpoint="/v1/chat/completions",
#   completion_window="24h",
#   metadata={
#     "description": "story from class batch"
#   }
# )

print(job_info)